In [ ]:
# LLM Setting 및 필수 라이브러리
import sys
import os
import time
import json
from tqdm import tqdm
from typing import List, Dict, Tuple

# 프로젝트 루트 경로 추가
sys.path.append(os.path.join(os.getcwd(), 'krx_llm_dataset'))

from utils import (
    get_law_text, process_law_data, make_index, 
    mcqa_graph, qa_graph, show_sample, show_spec,
    read_jsonl, write_jsonl
)
import pandas as pd

# 로컬 LLM 모델 경로 설정 (필요시 수정)
# 기본값: "/workspace/models/exaone-4.0-32b"
# 실제 모델 경로에 맞게 수정하세요
LOCAL_MODEL_PATH = os.getenv("LOCAL_MODEL_PATH", "/workspace/models/exaone-4.0-32b")

# 체크포인트 및 배치 설정
CHECKPOINT_DIR = "../data/law/checkpoints"
BATCH_SIZE = 5  # 로컬 LLM이므로 작은 배치 크기 사용
SAVE_INTERVAL = 10  # 10개마다 중간 저장

# 체크포인트 디렉토리 생성
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("✅ 모듈 로딩 완료")
print(f"📍 로컬 LLM 모델 경로: {LOCAL_MODEL_PATH}")
print(f"📁 체크포인트 디렉토리: {CHECKPOINT_DIR}")
print(f"⚙️ 배치 크기: {BATCH_SIZE}, 저장 간격: {SAVE_INTERVAL}")
print("⚠️ 모델 경로가 올바른지 확인하세요!")


In [ ]:
# 1. 법률 데이터 읽기 및 전처리

# 법률 JSON 파일 경로 설정
law_file_path = "../data/law/selected_laws.json"  # 법률 목록 파일
# law_file_path = "../data/law/law_sample.json"  # 상세 법령 내용 파일 (선택적으로 사용)

print(f"📖 법률 데이터 파일: {law_file_path}")

# 법률 데이터 읽기
try:
    law_data = get_law_text(law_file_path)
    print(f"✅ 법률 데이터 로딩 완료: {len(law_data)}개 항목")
    
    # 샘플 데이터 확인
    if len(law_data) > 0:
        print("\n📋 첫 번째 항목 샘플:")
        print(f"제목: {law_data[0]['title']}")
        print(f"내용: {law_data[0]['contents'][:200]}...")
        
except Exception as e:
    print(f"❌ 법률 데이터 로딩 실패: {e}")
    print("파일 경로를 확인하세요!")
    law_data = []


In [ ]:
# 2. 배치 처리 및 체크포인트 시스템 정의

def load_checkpoint(checkpoint_file: str) -> Dict:
    """체크포인트 파일을 로드합니다."""
    if os.path.exists(checkpoint_file):
        try:
            with open(checkpoint_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        except Exception as e:
            print(f"체크포인트 로드 오류: {e}")
    return {"completed_batches": [], "results": [], "failed_items": []}

def save_checkpoint(checkpoint_file: str, data: Dict):
    """체크포인트 파일을 저장합니다."""
    try:
        data['timestamp'] = time.time()
        data['formatted_time'] = time.strftime('%Y-%m-%d %H:%M:%S')
        with open(checkpoint_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        return True
    except Exception as e:
        print(f"체크포인트 저장 오류: {e}")
        return False

def create_batches(data: List[Dict], batch_size: int) -> List[List[Dict]]:
    """데이터를 배치 단위로 분할합니다."""
    batches = []
    for i in range(0, len(data), batch_size):
        batches.append(data[i:i + batch_size])
    return batches

def process_batch_with_graph(batch_data: List[Dict], task_type: str, eval_type: str, 
                            domain_type: str = "law", n_datasets: int = 2, max_step: int = 1) -> Tuple[List[Dict], List[str]]:
    """배치 데이터를 Graph Pipeline으로 처리합니다."""
    
    # Graph Pipeline 설정
    inputs = {
        "save_dir": CHECKPOINT_DIR,  # 임시 저장용
        "save_file_name": f"batch_{task_type}_{eval_type}_{int(time.time())}",
        "task_type": task_type,
        "eval_type": eval_type,
        "domain_type": domain_type,
        "n_datasets": n_datasets,
        "max_step": max_step,
        "n_workers": 3,  # 로컬 LLM이므로 낮게 설정
        "oai_model": "exaone-4.0-32b",
        "error_tolerance_ratio": 0.5,  # 높은 오류 허용도
        "show_log_error": True,
        "data": batch_data
    }
    
    try:
        # Graph 실행
        if eval_type == "mcqa":
            graph_instance = mcqa_graph()
        else:  # qa
            graph_instance = qa_graph()
            
        result = graph_instance.invoke(
            inputs=inputs, 
            config={"recursion_limit": 30}
        )
        
        # 결과 파일 읽기
        result_file = os.path.join(CHECKPOINT_DIR, f"{inputs['save_file_name']}_{task_type}_{eval_type}_{domain_type}_step_{max_step}.jsonl")
        
        if os.path.exists(result_file):
            results = read_jsonl(result_file)
            # 임시 파일 삭제
            os.remove(result_file)
            return results, []
        else:
            print(f"⚠️ 결과 파일을 찾을 수 없습니다: {result_file}")
            return [], [item['index'] for item in batch_data]
            
    except Exception as e:
        print(f"배치 처리 오류: {e}")
        return [], [item['index'] for item in batch_data]

print("✅ 배치 처리 시스템 준비 완료!")


In [ ]:
# 3. 데이터 전처리 및 배치 생성

# 품질 필터링 적용
try:
    filtered_data = process_law_data(
        law_file_path=law_file_path,
        title="금융법률",
        if_filter_punctuation=True,
        filter_punctuation_ratio=0.7,  # 법률 문서는 구둣점이 많으므로 비율 높임
        if_filter_english=True,
        filter_english_ratio=0.5,
        if_filter_number=True,
        filter_number_ratio=0.6,  # 법률 문서는 숫자(조항 번호 등)가 많으므로 비율 높임
        if_remove_unicode=True,
        if_normalize=True,
        token_threshold=100,  # 법률 조항은 짧을 수 있으므로 임계값 낮춤
        return_type="split",
        chunk_size=10000,
    )
    
    print(f"✅ 품질 필터링 완료: {len(filtered_data)}개 항목")
    
except Exception as e:
    print(f"❌ 데이터 필터링 실패: {e}")
    filtered_data = law_data  # 필터링 실패시 원본 데이터 사용

# 테스트를 위해 일부 데이터만 사용 (전체 데이터 사용시 주석 처리)
# sample_size = 20  # 테스트용 샘플 크기
sample_size = None  # 전체 데이터 사용

if sample_size and len(filtered_data) > sample_size:
    sample_data = filtered_data[:sample_size]
    print(f"🔬 테스트를 위해 {sample_size}개 샘플 사용")
else:
    sample_data = filtered_data
    print(f"📊 전체 {len(sample_data)}개 데이터 사용")

# 인덱스 생성
indexed_data = make_index(sample_data, prefix="금융법률")
print(f"✅ 인덱스 생성 완료: {len(indexed_data)}개 항목")

# 배치 단위로 분할
data_batches = create_batches(indexed_data, BATCH_SIZE)
print(f"📦 {len(data_batches)}개 배치 생성 (배치 크기: {BATCH_SIZE})")

# 샘플 확인
if len(indexed_data) > 0:
    print(f"\n📋 첫 번째 항목:")
    print(f"인덱스: {indexed_data[0]['index']}")
    print(f"제목: {indexed_data[0]['title']}")
    print(f"내용 미리보기: {indexed_data[0]['contents'][:150]}...")


In [ ]:
# 4. 체크포인트 기반 배치 처리 함수

def process_dataset_with_checkpoints(data_batches: List[List[Dict]], task_name: str, 
                                   task_type: str, eval_type: str) -> List[Dict]:
    """체크포인트를 사용하여 데이터셋을 안전하게 처리합니다."""
    
    checkpoint_file = os.path.join(CHECKPOINT_DIR, f"{task_name}_checkpoint.json")
    
    # 체크포인트 로드
    checkpoint = load_checkpoint(checkpoint_file)
    completed_batches = set(checkpoint.get("completed_batches", []))
    all_results = checkpoint.get("results", [])
    failed_items = checkpoint.get("failed_items", [])
    
    start_batch = len(completed_batches)
    total_batches = len(data_batches)
    
    print(f"🚀 {task_name} 처리 시작: {start_batch}/{total_batches} 배치부터")
    if start_batch > 0:
        print(f"📂 체크포인트에서 재시작: {len(all_results)}개 결과 로드됨")
    
    start_time = time.time()
    
    for batch_idx in tqdm(range(start_batch, total_batches), desc=f"🔄 {task_name} 배치 처리"):
        if batch_idx in completed_batches:
            continue
            
        batch_data = data_batches[batch_idx]
        print(f"\n📦 배치 {batch_idx + 1}/{total_batches} 처리 중... ({len(batch_data)}개 항목)")
        
        # 배치 처리
        batch_results, batch_failed = process_batch_with_graph(
            batch_data, task_type, eval_type
        )
        
        # 결과 추가
        all_results.extend(batch_results)
        failed_items.extend(batch_failed)
        completed_batches.add(batch_idx)
        
        print(f"✅ 배치 {batch_idx + 1} 완료: {len(batch_results)}개 성공, {len(batch_failed)}개 실패")
        
        # 체크포인트 저장
        checkpoint_data = {
            "completed_batches": list(completed_batches),
            "results": all_results,
            "failed_items": failed_items,
            "progress": f"{len(completed_batches)}/{total_batches}",
            "success_count": len(all_results),
            "failed_count": len(failed_items)
        }
        
        if save_checkpoint(checkpoint_file, checkpoint_data):
            print(f"💾 체크포인트 저장됨: {len(all_results)}개 결과")
        
        # 진행률 및 예상 완료시간 표시
        elapsed = time.time() - start_time
        if elapsed > 0:
            speed = (len(completed_batches) - start_batch) / elapsed
            remaining_batches = total_batches - len(completed_batches)
            eta_seconds = remaining_batches / speed if speed > 0 else 0
            eta_minutes = eta_seconds / 60
            
            print(f"⚡ 진행률: {len(completed_batches)}/{total_batches} ({len(completed_batches)/total_batches*100:.1f}%)")
            print(f"🕒 속도: {speed:.2f} 배치/초, 예상 완료: {eta_minutes:.1f}분 후")
        
        # 메모리 정리를 위한 잠시 대기
        time.sleep(2)
    
    total_time = time.time() - start_time
    print(f"\n🎉 {task_name} 완료!")
    print(f"📊 총 결과: {len(all_results)}개 성공, {len(failed_items)}개 실패")
    print(f"⏱️ 소요시간: {total_time/60:.1f}분")
    
    # 완료 후 체크포인트 파일 삭제
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
        print("🗑️ 체크포인트 파일 정리 완료")
    
    return all_results

print("✅ 체크포인트 기반 배치 처리 함수 준비 완료!")


In [ ]:
# 5. MCQA 데이터셋 배치 생성

print("🚀 MCQA 데이터셋 배치 생성 시작...")

try:
    mcqa_results = process_dataset_with_checkpoints(
        data_batches=data_batches,
        task_name="mcqa_generation",
        task_type="knowledge",
        eval_type="mcqa"
    )
    
    print(f"✅ MCQA 생성 완료: {len(mcqa_results)}개 문항")
    
    # 결과를 임시 저장
    mcqa_temp_file = os.path.join(CHECKPOINT_DIR, "mcqa_results_temp.jsonl")
    write_jsonl(mcqa_temp_file, mcqa_results)
    print(f"💾 MCQA 임시 저장: {mcqa_temp_file}")
    
except Exception as e:
    print(f"❌ MCQA 생성 실패: {e}")
    print("로컬 LLM 모델 경로와 설정을 확인하세요.")
    mcqa_results = []


In [ ]:
# 6. QA 데이터셋 배치 생성

print("🚀 QA 데이터셋 배치 생성 시작...")

try:
    qa_results = process_dataset_with_checkpoints(
        data_batches=data_batches,
        task_name="qa_generation",
        task_type="knowledge",
        eval_type="qa"
    )
    
    print(f"✅ QA 생성 완료: {len(qa_results)}개 문항")
    
    # 결과를 임시 저장
    qa_temp_file = os.path.join(CHECKPOINT_DIR, "qa_results_temp.jsonl")
    write_jsonl(qa_temp_file, qa_results)
    print(f"💾 QA 임시 저장: {qa_temp_file}")
    
except Exception as e:
    print(f"❌ QA 생성 실패: {e}")
    print("로컬 LLM 모델 경로와 설정을 확인하세요.")
    qa_results = []


In [ ]:
# 7. 생성된 데이터셋 확인 및 CSV 변환

print("📊 생성된 데이터셋 확인 및 CSV 변환...")

def convert_options_to_numbers(options_list):
    """선택지를 숫자 형태로 변환 (data_download.ipynb 형식에 맞춤)"""
    converted = []
    for i, option in enumerate(options_list):
        converted.append(f"{i+1}. {option}")
    return converted

# MCQA 결과 처리
mcqa_df = None
if mcqa_results:
    try:
        mcqa_df = pd.DataFrame(mcqa_results)
        print(f"✅ MCQA 데이터프레임 생성: {len(mcqa_df)}개 문항")
        
        # 샘플 확인
        if len(mcqa_df) > 0:
            print("\n🔍 MCQA 샘플:")
            show_sample(mcqa_df, n=1)
            
            # CSV 변환 (data_download.ipynb 형식)
            mcqa_csv_path = "../data/law/mcqa.csv"
            
            if all(col in mcqa_df.columns for col in ['question', 'options', 'answer']):
                mcqa_clean = mcqa_df.copy()
                
                # options를 숫자 형태로 변환
                mcqa_clean['options'] = mcqa_clean['options'].apply(
                    lambda x: convert_options_to_numbers(x) if isinstance(x, list) else []
                )
                
                # Question 컬럼: 문제 + 선택지
                mcqa_clean["Question"] = mcqa_clean["question"] + "\n" + mcqa_clean["options"].apply(
                    lambda x: "\n".join(x) if x else ""
                )
                
                # Answer 컬럼: "답변: 숫자" 형태
                answer_mapping = {"A": "1", "B": "2", "C": "3", "D": "4", "E": "5", "F": "6", "G": "7"}
                mcqa_clean["Answer"] = "답변: " + mcqa_clean["answer"].map(answer_mapping).fillna(mcqa_clean["answer"])
                
                # Question, Answer 컬럼만 선택
                final_mcqa = mcqa_clean[["Question", "Answer"]]
                
                # CSV 저장
                final_mcqa.to_csv(mcqa_csv_path, index=False, encoding='utf-8-sig')
                print(f"💾 MCQA CSV 저장 완료: {mcqa_csv_path}")
                print(f"📊 MCQA 최종 형식: Question, Answer 컬럼 ({len(final_mcqa)}개 문항)")
            else:
                print("⚠️ MCQA 데이터에 필요한 컬럼이 없습니다.")
                
    except Exception as e:
        print(f"❌ MCQA 데이터 처리 실패: {e}")
else:
    print("❌ MCQA 결과가 없습니다.")

print("\n" + "="*80)

# QA 결과 처리
qa_df = None
if qa_results:
    try:
        qa_df = pd.DataFrame(qa_results)
        print(f"✅ QA 데이터프레임 생성: {len(qa_df)}개 문항")
        
        # 샘플 확인
        if len(qa_df) > 0:
            print("\n🔍 QA 샘플:")
            for i in range(min(2, len(qa_df))):
                print(f"\n--- QA {i+1} ---")
                print(f"질문: {qa_df.iloc[i]['question']}")
                print(f"답변: {qa_df.iloc[i]['answer'][:200]}...")
                
            # CSV 변환 (data_download.ipynb 형식)
            qa_csv_path = "../data/law/qa.csv"
            
            if all(col in qa_df.columns for col in ['question', 'answer']):
                qa_clean = qa_df.copy()
                
                # Question 컬럼: 질문 그대로
                qa_clean["Question"] = qa_clean["question"]
                
                # Answer 컬럼: "답변: " + 답변
                qa_clean["Answer"] = "답변: " + qa_clean["answer"].astype(str)
                
                # Question, Answer 컬럼만 선택
                final_qa = qa_clean[["Question", "Answer"]]
                
                # CSV 저장
                final_qa.to_csv(qa_csv_path, index=False, encoding='utf-8-sig')
                print(f"💾 QA CSV 저장 완료: {qa_csv_path}")
                print(f"📊 QA 최종 형식: Question, Answer 컬럼 ({len(final_qa)}개 문항)")
            else:
                print("⚠️ QA 데이터에 필요한 컬럼이 없습니다.")
                
    except Exception as e:
        print(f"❌ QA 데이터 처리 실패: {e}")
else:
    print("❌ QA 결과가 없습니다.")

print("\n✅ 데이터셋 생성 및 CSV 변환 완료!")


In [ ]:
# 8. 최종 CSV 파일 확인 및 정리

print("📋 생성된 CSV 파일 확인...")

# CSV 파일 경로
mcqa_csv_path = "../data/law/mcqa.csv"
qa_csv_path = "../data/law/qa.csv"

# MCQA CSV 확인
if os.path.exists(mcqa_csv_path):
    print(f"✅ MCQA CSV 파일 생성됨: {mcqa_csv_path}")
    mcqa_csv_df = pd.read_csv(mcqa_csv_path)
    print(f"📊 MCQA 데이터 크기: {len(mcqa_csv_df)}개 문항")
    
    # 컬럼 정보 출력
    print(f"📋 MCQA 컬럼: {list(mcqa_csv_df.columns)}")
    
    # 첫 번째 행 미리보기
    if len(mcqa_csv_df) > 0:
        print("\n🔍 MCQA CSV 첫 번째 문항:")
        print(f"Question: {mcqa_csv_df.iloc[0]['Question'][:200]}...")
        print(f"Answer: {mcqa_csv_df.iloc[0]['Answer']}")
else:
    print(f"❌ MCQA CSV 파일이 없습니다: {mcqa_csv_path}")

print("\n" + "-"*60)

# QA CSV 확인
if os.path.exists(qa_csv_path):
    print(f"✅ QA CSV 파일 생성됨: {qa_csv_path}")
    qa_csv_df = pd.read_csv(qa_csv_path)
    print(f"📊 QA 데이터 크기: {len(qa_csv_df)}개 문항")
    
    # 컬럼 정보 출력
    print(f"📋 QA 컬럼: {list(qa_csv_df.columns)}")
    
    # 첫 번째 행 미리보기
    if len(qa_csv_df) > 0:
        print("\n🔍 QA CSV 첫 번째 문항:")
        print(f"Question: {qa_csv_df.iloc[0]['Question']}")
        print(f"Answer: {qa_csv_df.iloc[0]['Answer'][:200]}...")
else:
    print(f"❌ QA CSV 파일이 없습니다: {qa_csv_path}")

# 임시 파일 정리
temp_files = [
    os.path.join(CHECKPOINT_DIR, "mcqa_results_temp.jsonl"),
    os.path.join(CHECKPOINT_DIR, "qa_results_temp.jsonl")
]

for temp_file in temp_files:
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"🗑️ 임시 파일 삭제: {temp_file}")

print("\n" + "="*80)
print("🎉 법률 QA/MCQA 데이터셋 생성 완료!")
print(f"📁 최종 출력 파일 (Question, Answer 컬럼):")
print(f"  - MCQA: {mcqa_csv_path}")
print(f"  - QA: {qa_csv_path}")
print("✨ data_download.ipynb 형식에 맞춘 CSV 파일이 생성되었습니다!")
print("🔄 배치 처리 및 체크포인트 시스템으로 안전하게 처리되었습니다.")
